**NOTE**: Our primary objective was **Crop Yield Prediction** through the **integration of Machine Learning and DSSAT**. However, **due to limited dataset, the DSSAT conditions could not be fully satisfied**. The code has been developed using Machine Learning, and **it remains entirely feasible** to extend this work into a **hybrid model combining Machine Learning with DSSAT** for enhanced prediction accuracy.

By: **Akhilesh Pant** (MCA) ~ **Amrapali University**, Haldwani


**Project**: Crop Yield Prediction using Machine Learning (XGBoost)
**Goal**   : Train a regression model on historical crop data and
         predict yield (in quintals/ha) from Area and Production.

Dataset columns expected (from your original CSV):
- 'Year'
- 'Area (Hectare)'           -> numeric with commas (e.g., "1,00,560.00")
- 'Production (Tonnes)'      -> numeric with commas
- 'Yield (Tonne/Hectare)'    -> numeric with commas

Key ideas:
- We clean numbers (remove commas), convert Yield to quintals/ha.
- Features (X): Area & Production
- Target (y): log(1 + Yield_Quintal_per_ha) for stable learning
- Model: XGBoost Regressor (tree-based ML)
- Metrics: RMSE, R², MAPE, and derived "Accuracy" = 100 - MAPE
"""


# IMPORT LIBRARIES


In [89]:
import pandas as pd   # data loading/manipulation
import numpy as np   # numerical ops (log, arrays)
import joblib         # save/load trained model artifacts

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, root_mean_squared_error
import xgboost as xgb   # gradient-boosted trees for regression


# Step 1: Load and Clean Data

In [91]:
df = pd.read_csv("usn_rice_kharif_2003-2023.csv")

# **Note**: Given dataset stores numbers with commas. E.g: "1,00, 560.00". We need to remove commas and convert to float so the model can use them. ~ Akhilesh Pant

**That block of code is doing two things:**

1. **Remove commas from numbers stored as strings**

Example: "1,234" → "1234"

This is needed because numbers in CSV/Excel files often come with commas as thousand separators.

2. **Convert the cleaned string values into floats**

Example: "1234" → 1234.0 """

In [96]:
for col in ["Area (Hectare)", "Production (Tonnes)", "Yield (Tonne/Hectare)"]:
    df[col] = (
        df[col]
       .astype(str) # Ensure string (so we can call .str methods)
       .str.replace(",", "") # Remove thousands sepeartors 
       .astype(float) # Convert clean strings to float
    ) 

In [98]:
# Convert Yield units for the target
# Original: tones/hectare
# Target: quintals/hectare  (1 tonne = 10 quintals)

df["Yield_Quintal_per_ha"]= df["Yield (Tonne/Hectare)"]

**Choose featues (inputs) and target (output) for the model**

# We will use only Area and Production as predictors to keep it aligned with the given dataset.

In [102]:
x = df[["Area (Hectare)", "Production (Tonnes)"]]

**We use log1p because it makes very big numbers smaller, reduces the effect of outliers, and helps the regression model learn better patterns.**

In [105]:
# For regression stability we use a long-transdorm on the target:
# y = log(1 + Yield_Quintal_per_ha)

# This reduces the effect of the large values and helps the helps the model learn smoother patterns.

y = np.log1p(df["Yield_Quintal_per_ha"])

In [107]:
scaler = StandardScaler()
x_scaled = scaler.fit_transform(x)

```python
from sklearn.preprocessing import StandardScaler  

scaler = StandardScaler()  
X_scaled = scaler.fit_transform(X)  
```

### Why use `StandardScaler()`?

`StandardScaler()` standardizes features by removing the mean and scaling to unit variance:

$$
X_{scaled} = \frac{X - \mu}{\sigma}
$$

* **$\mu$** = Mean of the feature
* **$\sigma$** = Standard deviation of the feature

This ensures all features are on the same scale, improving the performance of algorithms like Logistic Regression, SVM, KNN, and Neural Networks.




**Save a cleaned version of datase** 

In [120]:
cleaned_path = "cleaned_crop_data.csv"
df.to_csv(cleaned_path, index = False)
print(f" Cleaned dataset saved as {cleaned_path}")

 Cleaned dataset saved as cleaned_crop_data.csv


# Step 2: Train,  Test & Split

**Split the data into training (80%) & testing (20%).**

The test set simulates "unseen" data to evaluate generalization.

In [123]:
x_train, x_test, y_train, y_test = train_test_split(
    x_scaled,    # scaled features
    y,           # log-transformed target
    test_size=0.2, # 20% test split 
    random_state=42  # fixed seed for reproducibility
)

# Step 3: Train the Model

In [126]:
# Define an XGBoost regressor. Key hyperparameters:
# - learning_rate: smaller -> slower but often better generalization
# - max_depth    : depth of individual trees (controls complexity)
# - n_estimators : number of trees in the ensemble
# - subsample    : stochasticity to reduce overfitting

model = xgb.XGBRegressor(
    objective="reg:squarederror",
    learning_rate=0.05,
    max_depth=5,
    n_estimators=100,
    subsample=0.8,
    random_state=42
)

# Fit the model on the training data
model.fit(x_train,  y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

**We created an XGBoost regression model and then trained it using your training dataset so it can learn patterns and later make predictions.**


---

```python
model = xgb.XGBRegressor(
    objective="reg:squarederror",
    learning_rate=0.05,
    max_depth=5,
    n_estimators=100,
    subsample=0.8,
    random_state=42
)
```

1. **`model = xgb.XGBRegressor(...)`**
   → Creates an **XGBoost regression model** object with the parameters given below.

2. **`objective="reg:squarederror"`**
   → Sets the model’s goal to **regression** (predicting continuous values) and uses **squared error loss** for measuring errors.

3. **`learning_rate=0.05`**
   → Controls how fast the model learns. A smaller value (0.05) means **slower but more accurate learning**.

4. **`max_depth=5`**
   → Limits the **depth of each decision tree**. Prevents trees from becoming too complex and overfitting.

5. **`n_estimators=100`**
   → Number of **trees (boosting rounds)** to be built. Here, the model will train **100 trees** sequentially.

6. **`subsample=0.8`**
   → Uses **80% of the training data randomly** for each tree, which helps prevent overfitting.

7. **`random_state=42`**
   → Fixes the randomness (seed), so results are **reproducible** every time you run the code.

---

```python
# Fit the model on the training data
model.fit(x_train, y_train)
```

8. **`model.fit(x_train, y_train)`**
   → Trains the XGBoost model on the **features (`x_train`)** and **target values (`y_train`)**.
   The model learns patterns from the training data to make predictions later.

---


# Step 4: Evaluate

**Predict log-space values on the test set**

In [132]:
y_pred_log = model.predict(x_test)

**Inverse the log-transform to get predictions back in original units (quintals/ha)**

In [135]:
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)


```python
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)
```

---

### 🔹 Step 1: `np.expm1()`

* `np.expm1(x)` means **exp(x) - 1**.

* It is mathematically equivalent to:

  $$
  \text{expm1}(x) = e^x - 1
  $$

* It’s often used when your target values were **log-transformed** earlier for modeling.
  (Because `np.log1p(x)` = log(x + 1) is commonly used to handle skewed data → then later you reverse it with `np.expm1`.)

---

### 🔹 Step 2: `y_pred_log`

* This is your **model’s prediction in log scale** (because you trained on `log(y + 1)` instead of raw `y`).
* After prediction, you must convert it **back to the original scale**.
* That’s why you use:

  ```python
  y_pred = np.expm1(y_pred_log)
  ```

This reverses the earlier `np.log1p()` transformation and gives predictions in the **real/original units**.

---

### 🔹 Step 3: `y_test`

* Your test labels (`y_test`) were also log-transformed earlier.
* To compare predictions with actual values, you need to bring them back to the original scale.

  ```python
  y_true = np.expm1(y_test)
  ```

---

**Why do we do this?**

* Many real-world target variables (like prices, salaries, sales) are **heavily skewed**.
* Taking `log1p()` makes the data more **normal (Gaussian-like)** → better for regression models.
* After training & prediction, you must reverse it using `expm1()` so you can interpret results correctly.

---

 **Example**:

Suppose your dataset had a house price `y = 199999`.

* Before training:

  ```python
  y_log = np.log1p(199999)   # ≈ 12.206
  ```

* Model predicts something like `12.20`.

* After prediction:

  ```python
  y_pred = np.expm1(12.20)   # ≈ 199999 (back to rupees/dollars)
  ```

---


In [145]:
# Compute evaluation metrics:
#   RMSE: root mean squared error (same units as target: quintals/ha)
#   R²  : coefficient of determination (how much variance is explained)
#   MAPE: mean absolute percentage error (lower is better)

rmse = root_mean_squared_error(y_true, y_pred)  # No warning now
r2 = r2_score(y_true, y_pred)
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100


---

We are comparing the **actual values** (`y_true`) with the **predicted values** (`y_pred`).
The goal: measure how good your model is.

---

### 1. RMSE (Root Mean Squared Error)

```python
rmse = root_mean_squared_error(y_true, y_pred)
```

This tells you the **average size of the errors** in the same units as the target.

* It squares errors (so negatives don’t cancel out), averages them, then takes a square root.
* Smaller RMSE = better model.

---

### 2. R² (Coefficient of Determination)

```python
r2 = r2_score(y_true, y_pred)
```

This shows **how much of the variation in the actual data is explained by the predictions**.

* R² = 1 → perfect prediction.
* R² = 0 → model is no better than always predicting the average.
* R² < 0 → model is worse than predicting the average.

---

### 3. MAPE (Mean Absolute Percentage Error)

```python
mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
```

This shows the **average error as a percentage of actual values**.
Step by step:

* `y_true - y_pred`: error for each point.
* `abs(...)`: ignore sign.
* Divide by `y_true`: turn error into a percentage.
* `mean`: average it.
* Multiply by 100: express as a percentage.

Example: if MAPE = 8, then predictions are off by about 8% on average.

---

In short:

* **RMSE → average error size (in real units).**
* **R² → how well predictions fit actual data.**
* **MAPE → average percentage error.**



# **A simple "accuracy" proxy often shown with MAPE:**

In [149]:
accuracy = 100 - mape

print(f"\n📊 RMSE: {rmse: .2f}")
print(f"📈 R²: {r2:.4f}")
print(f"📉 MAPE: {mape:.2f}%")
print(f"✅ Accuracy: {accuracy:.2f}%") 


📊 RMSE:  0.22
📈 R²: 0.7305
📉 MAPE: 7.34%
✅ Accuracy: 92.66%



```python
accuracy = 100 - mape
```

* **MAPE (Mean Absolute Percentage Error)** tells you how far off your predictions are in percentage terms (lower is better).
* Accuracy here is defined as `100 - MAPE`, meaning if your error is 8%, then your accuracy is **92%**.

```python
print(f"\nRMSE: {rmse:.2f}")
```

* **RMSE (Root Mean Squared Error)** shows the average size of prediction errors in the same unit as your target variable.
* The lower RMSE is, the better the model fits.
* `:.2f` means it will print only 2 decimal places.

```python
print(f"R²: {r2:.4f}")
```

* **R² (Coefficient of Determination)** explains how much of the variation in the target variable is explained by the model.
* Value ranges from 0 to 1 (sometimes negative if model is bad).
* Closer to **1.0** → better performance.
* `:.4f` means 4 decimal places.

```python
print(f"MAPE: {mape:.2f}%")
```

* Shows the **average error** in percentage terms.
* Example: if `MAPE = 6.35`, it means predictions are off by \~6.35% on average.

```python
print(f"Accuracy: {accuracy:.2f}%")
```

* Prints the **model’s accuracy** (100 - MAPE) with 2 decimal places.

---

# **In short:**

* **RMSE** = how large your prediction errors are (scale of target).
* **R²** = how much variation is explained by the model.
* **MAPE** = average percentage error.
* **Accuracy** = 100 - MAPE (percentage correctness).



# Step 5: Save Artifacts

In [153]:
# Persist the trained model and the scaler so you can load them later for prediction

joblib.dump(model, "Crop_Predict_model.pkl")
joblib.dump(scaler, "Crop_scaler.pkl")

print("📂 Model and scaler saved successfully.")


📂 Model and scaler saved successfully.


# Step 6: Prediction Function

In [174]:
def predict_crop_yield(area, production):
    """
    Predict yield (quintals/ha) from Area (Hectare) and Production (Tonnes).
    """
    # Load the trained model and scaler
    model = joblib.load("Crop_Predict_model.pkl")
    scaler = joblib.load("Crop_scaler.pkl")

    # Prepare input data
    X_input = pd.DataFrame([[area, production]], 
                           columns=["Area (Hectare)", "Production (Tonnes)"])

    # Apply the SAME scaling used during training
    X_scaled = scaler.transform(X_input)

    # Predict log-space value, then invert to original units
    y_pred_log = model.predict(X_scaled)[0]
    predicted_yield = np.expm1(y_pred_log)

    print(f"\nEstimated Yield: {predicted_yield:.2f} quintals/ha")
    return predicted_yield


# Step 7: Example Usage (only runs when this file is executed directly)

In [177]:
if __name__ == "__main__":
    # Example: Area = 100,000 ha, Production = 270,000 tonnes
    # This will print an estimated yield in quintals/ha
    predict_crop_yield(100000, 270000)


Estimated Yield: 2.84 quintals/ha
